# Laboratorio 3

En este laboratorio nuevamente utilizaremos los datos de COVID-19 disponibilizados por el Ministerio de Ciencias, Tecnología, Conocimiento e Innovación de Chile ([info](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto1)).

In [3]:
import pandas as pd

covid = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19_std.csv")
covid.head()

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,Fecha,Casos confirmados
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-03-30,6.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-03-30,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-03-30,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-03-30,0.0
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,2020-03-30,NaN


## Ejercicio 1

(1 pto)

¿Cuál es la región que tiene menos registros (filas)?

In [185]:
dp=covid.pivot_table(index=['Region'], aggfunc='size')
dp.idxmin()

'Arica y Parinacota'

In [87]:
cont=covid['Region'].value_counts()
cont.idxmin()

'Arica y Parinacota'

__Respuesta:__ La región de Arica y Parinacota

## Ejercicio 2

(2 puntos)

1. Define la serie `casos` tal que:
    * Solo sean los registros de la región Valparaíso.
    * Los _index_ deben ser la _Fecha_ pero en formato `datetime`. Hint: `pd.to_datetime()`
    * Los _values_ deben ser los _Casos confirmados_.
2. Define la serie `casos_not_null` filtrando los calores `NaN` de la serie `casos`.
3. Obtén el mínimo, máximo y suma y promedio de la serie `casos_not_null`.

In [118]:
covid_valpo = covid.query("Region =='Valparaíso'")
covid_valpo.loc[:,'Fecha'], covid_valpo

(49       2020-03-30
 50       2020-03-30
 51       2020-03-30
 52       2020-03-30
 53       2020-03-30
             ...    
 19631    2020-09-28
 19632    2020-09-28
 19633    2020-09-28
 19634    2020-09-28
 19635    2020-09-28
 Name: Fecha, Length: 2145, dtype: object,
            Region  Codigo region                  Comuna  Codigo comuna  \
 49     Valparaíso              5               Algarrobo         5602.0   
 50     Valparaíso              5                 Cabildo         5402.0   
 51     Valparaíso              5                  Calera         5502.0   
 52     Valparaíso              5             Calle Larga         5302.0   
 53     Valparaíso              5               Cartagena         5603.0   
 ...           ...            ...                     ...            ...   
 19631  Valparaíso              5              Valparaiso         5101.0   
 19632  Valparaíso              5           Villa Alemana         5804.0   
 19633  Valparaíso              5         

In [100]:
index=covid_valpo.loc[:,'Fecha']
indices = pd.to_datetime(covid_valpo.loc[:,'Fecha'], format='%Y-%m-%d')
indices

49      2020-03-30
50      2020-03-30
51      2020-03-30
52      2020-03-30
53      2020-03-30
           ...    
19631   2020-09-28
19632   2020-09-28
19633   2020-09-28
19634   2020-09-28
19635   2020-09-28
Name: Fecha, Length: 2145, dtype: datetime64[ns]

In [121]:
covid_valpo.loc[:,'Casos confirmados']

49          7.0
50          0.0
51          6.0
52          0.0
53          0.0
          ...  
19631    7020.0
19632    2583.0
19633    6701.0
19634     141.0
19635     449.0
Name: Casos confirmados, Length: 2145, dtype: float64

In [130]:
covid_valpo = covid.query("Region =='Valparaíso'")
indices= pd.to_datetime(covid_valpo.loc[:,'Fecha'], format='%Y-%m-%d')
values = covid_valpo.loc[:,'Casos confirmados'].values
casos = pd.Series(data=values, index=indices)
casos.head()

Fecha
2020-03-30    7.0
2020-03-30    0.0
2020-03-30    6.0
2020-03-30    0.0
2020-03-30    0.0
dtype: float64

In [131]:
casos_not_null = casos.dropna()
casos_not_null

Fecha
2020-03-30       7.0
2020-03-30       0.0
2020-03-30       6.0
2020-03-30       0.0
2020-03-30       0.0
               ...  
2020-09-28    7020.0
2020-09-28    2583.0
2020-09-28    6701.0
2020-09-28     141.0
2020-09-28     449.0
Length: 2118, dtype: float64

In [135]:
casos_min = casos_not_null.min()
casos_max = casos_not_null.max()
casos_sum = casos_not_null.sum()
casos_mean = casos_not_null.mean()

(790461.0, 373.2110481586402)

In [136]:
print(f"Mínimo casos: {casos_min}")
print(f"Máximo casos: {casos_max}")
print(f"Suma de casos: {casos_sum}")
print(f"Promedio de casos: {casos_mean}")

Mínimo casos: 0.0
Máximo casos: 7020.0
Suma de casos: 790461.0
Promedio de casos: 373.2110481586402


## Ejercicio 3

* Define el dataframe  `covid_full` uniendo (`merge`) los dataframes `covid_total` y `covid_death` utilizando la _Fecha_ como _key_ y de tal forma de unir solo los registros que se encuentren en ambos conjuntos de datos. Hint: Escoger sabiamente entre _left_, _right_, _inner_ u _outer join_.
* Filtra el dataframe `covid_full` tal que:
    - Fechas con por lo menos 1000 casos totales.
    - Fechas con por lo menos 100 casos nuevos con síntomas.
    - Fechas con más de 20 muertes en el grupo etárea `70-79`.

¿Cuál es la fecha máxima y mínima? ¿Cuántos registros quedan finalmente?

In [4]:
covid_total = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv", usecols=range(3))
covid_total.head()

,Fecha,Casos nuevos con sintomas,Casos totales
0,2020-03-02,1.0,1.0
1,2020-03-03,0.0,1.0
2,2020-03-04,2.0,3.0
3,2020-03-05,1.0,4.0
4,2020-03-06,1.0,5.0


In [5]:
covid_death = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario_T.csv").rename(columns={"Grupo de edad": "Fecha"})
covid_death.head()

,Fecha,<=39,40-49,50-59,60-69,70-79,80-89,>=90
0,2020-04-09,2,1,1,9,21,19,4
1,2020-04-10,2,2,2,10,22,21,6
2,2020-04-11,3,2,2,12,24,24,6
3,2020-04-12,3,2,3,12,28,25,7
4,2020-04-13,3,2,3,12,29,26,7


In [6]:
covid_full = pd.merge(covid_total, covid_death, how= 'inner', on=['Fecha'])
covid_full.head()

,Fecha,Casos nuevos con sintomas,Casos totales,<=39,40-49,50-59,60-69,70-79,80-89,>=90
0,2020-04-09,426.0,5972.0,2,1,1,9,21,19,4
1,2020-04-10,529.0,6501.0,2,2,2,10,22,21,6
2,2020-04-11,426.0,6927.0,3,2,2,12,24,24,6
3,2020-04-12,286.0,7213.0,3,2,3,12,28,25,7
4,2020-04-13,312.0,7525.0,3,2,3,12,29,26,7


In [10]:
mask1 = covid_full[covid_full.loc[:,'Casos totales'].values>=1000]
mask2 = covid_full[covid_full.loc[:,'Casos nuevos con sintomas'].values>=100]
mask3 = covid_full[covid_full.loc[:,'70-79'].values>20]

mask1, mask2, mask3 =covid_full_filtered1[:,'Casos totales']
covid_full_filtered = pd.merge(covid_full_filtered1, mask3, how='outer', on=['Fecha'])


AttributeError: 'function' object has no attribute 'duplicates'

In [215]:
covid_full_filtered.loc[0,'Fecha'], covid_full_filtered.loc[176, 'Fecha'], type(covid_full_filtered)

('2020-04-09', '2020-10-02', pandas.core.frame.DataFrame)

In [231]:
print(f"Fecha mínima: {covid_full_filtered.loc[0,'Fecha']}"),
print(f"Fecha máxima: {covid_full_filtered.loc[176,'Fecha']}"),
print(f"Cantidad de registros finalmente: {len(covid_full_filtered)-1}")

Fecha mínima: 2020-04-09
Fecha máxima: 2020-10-02
Cantidad de registros finalmente: 176
